In [1]:
import os

In [2]:
%pwd

'c:\\Users\\midem\\Blood-Cells-Classification-Using-CNN-DVC-Pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\midem\\Blood-Cells-Classification-Using-CNN-DVC-Pipeline'

In [5]:
# To be used to update entity (src/bloodcellClassifier/entity/config_entity.py)
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
# To be used to update configuration manager (src/bloodcellClassifier/config/configuration.py)
from bloodcellClassifier.constants import *
from bloodcellClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
# To be used to update the data ingestion component (src/bloodcellClassifier/components/
import os
import urllib.request as request
import zipfile
from bloodcellClassifier import logger
from bloodcellClassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
# To be used to update the data ingestion pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-21 22:08:41,678: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-21 22:08:41,681: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-21 22:08:41,683: INFO: common: created directory at: artifacts]
[2024-10-21 22:08:41,685: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-21 22:08:48,831: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Cache-Control: private
Content-Length: 276120
Content-Type: text/html; charset=utf-8
P3P: CP="ALL IND DSP COR ADM CONo CUR CUSo IVAo IVDo PSA PSD TAI TELo OUR SAMo CNT COM INT NAV ONL PHY PRE PUR UNI"
WWW-Authenticate: Wlid1.1 realm="WindowsLive", fault="BadContextToken", policy="MBI_SSL", ver="7.5.0.0", target="ssl.live.com", siteId="ssl.live.com"
X-NetworkStatistics: 0,525568,0,0,31452,0,83497,3
X-SharePointHealthScore: 1
Content-Security-Policy: frame-ancestors 'self' sentry.contentvalidation.com sentry.ppe.contentvalidation.com sentry.int.conte

BadZipFile: File is not a zip file